## [412周赛](https://leetcode.cn/contest/weekly-contest-412/)

### [统计近似相等数对 II](https://leetcode.cn/problems/count-almost-equal-pairs-ii/description/)
### 题目
给你一个正整数数组nums。如果我们执行以下操作至多两次可以让两个整数x和y相等，那么我们称这个数对是近似相等的：
- 选择x或者y之一，将这个数字中的两个数位交换。

请你返回nums中，下标i和j满足i < j且nums[i]和nums[j]近似相等的数对数目。注意执行操作后得到的整数可以有前导0 。


### 分析
- 枚举右, 维护左: 向右边遍历数组时使用哈希表维护index左边数据的状态
- 由于可以存在前导0, 因此我们有两种做法, 一种是在每个数上加上前导0, 还有一种是交换时我们只选择长度大的进行交换(因为长度大的可以通过与0换位置得到长度小的)。这里我们为了方便选择第二种处理方法(因为每个数加上前导0会对后面造成很多麻烦), 为了能在维护左边数据状态时一定是左边数的长度小于右边的, 因此我们需要根据长度将nums进行排序
- 此处我们维护左边数据状态的方法就是, 记录数字出现次数. 当我们遍历到numj时, 将numj经过之多两次交换能得到哪些数字用集合(用于去重)记录下来, 然后遍历集合, 看看左边的数据状态中这些数字出现了多少次, 我们的答案就需要+多少次
- 我们可以使用暴力的做法将至多交换两次可以得到的数据放到集合中(暴力做法中注意要复原现场)


In [ ]:
import collections

def process(nums):
    nums.sort(key=lambda x: len(str(x)))
    ans = 0
    cnt = collections.defaultdict(int)
    for num in nums:
        st = {num}  # 什么都不交换
        s = list(str(num))  # 得到数值列表, 方便交换
        m = len(s)
        # 暴力交换
        for i in range(m):
            for j in range(i+1, m):
                s[i], s[j] = s[j], s[i]   # 第一次交换
                st.add(int("".join(s)))

                for p in range(i+1, m):   # 这里从i+1开始是为了避免和第一次交换重复(不需要再从0开始交换)
                    for q in range(p+1, m):
                        s[p], s[q] = s[q], s[p]  # 第二次交换
                        st.add(int("".join(s)))
                        s[p], s[q] = s[q], s[p]  # 注意复原
                s[i], s[j] = s[j], s[i]  # 注意复原
        # 遍历左边数据状态得到答案
        ans += sum(cnt[v] for v in st)
        # 更改左边数据状态
        cnt[num] += 1   
    return ans
        

            


### [K 次乘运算后的最终数组 II](https://leetcode.cn/problems/final-array-state-after-k-multiplication-operations-ii/description/)
### 题目
给你一个整数数组nums ，一个整数k和一个整数multiplier 。你需要对nums执行k次操作，每次操作中：
- 找到 nums 中的 最小 值 x ，如果存在多个最小值，选择最 前面 的一个。
- 将 x 替换为 x * multiplier 。

k次操作以后，你需要将nums中每一个数值对 10^9 + 7 取余。请你返回最终的nums数组。(注意这里的k非常大)

### 分析
- 我们可以使用最小堆的方式, 这样可以更快的得到最小的那个数(使用heapq.heapreplace可以更高效的替换堆中最小元素). 但由于这里的K非常大, 因此我们要想办法让操作于K无关。
- 经过分析可以知道两个结论:
    - 1、当x,y,z很接近时, 一定是从小到大进行依次乘以m的, 根据这个规律我们可以直接得到每个元素还需要进行多少次的乘法: 如果排序后i < k%n, 则还需要执行 k//n + 1次; 如果i >= k%n, 则还需要执行k//n 次的乘法(因为此处i从0开始, 所以当k%n=0时是不需要多执行一次的)
    - 2、对于原始数组, 我们可以先使用循环最小堆的方式, 不断将数组中每个元素乘以m, 使得他们相互靠近, 当最后出堆的是num原始中的最大值时, 此时数组就满足了结论1的先决条件: 各数组都非常靠近
- 需要注意到一些特殊情况, 如果k比较小时可能在循环最小堆的时候就已经解决问题了, 因此循环最小堆的循环条件是k > 0 ; 当multiplier为1时, 是不可能退出得了循环最小堆的(因为最小是*1是不可能靠近最大值的)

In [ ]:
import heapq

def process(nums, k, multiplier):
    if multiplier == 1:
        return nums

    MOD = 1_000_000_000 + 7
    n = len(nums)
    mx = max(nums)
    h = [(num, i) for i, num in enumerate(nums)]  # 需要保留坐标
    heapq.heapify(h)

    # 在此处可以先不进行nums的变换
    while k > 0:
        num, i = h[0]  # 拿到堆中最小值
        heapq.heapreplace(h, (num*multiplier, i))   # 先pop再push, 更高效
        k -= 1         # 注意此处要将k减小
        if num == mx:  # 如果出堆的是原始数组中的最小值, 那么说明此时已经满足很靠近的条件了
            break
    
    for i in range(n):  # 这里的i表示就是此时num中数组排续的位次
        num, j = heapq.heappop(h)
        nums[j] = num * pow(multiplier, k//n + (i < k%n), MOD) % MOD   # 这里使用int + bool巧妙地进行了if操作
    return nums

## [416周赛](https://leetcode.cn/contest/weekly-contest-416/)

### [移山所需的最少秒数](https://leetcode.cn/problems/minimum-number-of-seconds-to-make-mountain-height-zero/description/)
### 题目
给你一个整数 mountainHeight 表示山的高度。同时给你一个整数数组 workerTimes，表示工人们的工作时间（单位：秒）。对于工人i: 把山的高度降低x，需要花费workerTimes[i] + workerTimes[i] * 2 +...+ workerTimes[i] * x秒。例如:
- 山的高度降低 1，需要 workerTimes[i] 秒。
- 山的高度降低 2，需要 workerTimes[i] + workerTimes[i] * 2 秒，依此类推。

返回一个整数，表示工人们使山的高度降低到0所需的最少秒数。注意: 工人们是同时进行工作的。

### 分析
- 由于工人们是同时进行工作, 所以得到每个工人工作时间列表后需要求最大值. 我们可以使用模拟堆的方式, 每次选一个「工作后总用时」最短的工人，把山的高度降低 1。因此我们的堆中元素为(工作后该员工总用时, 下一次工作的增量时间, 工人基础时间)
- 每次工人工作后维护当前最多用时

In [ ]:
import heapq
def process(mountainHeight: int, workerTimes):
    cost = [(num, 2*num, num) for num in workerTimes]
    heapq.heapify(cost)

    ans = 0
    for _ in range(mountainHeight):  # 循环山的高度, 每次挑选「工作后总用时」最短的工人
        time, delta, base = heapq.heappop(cost)
        ans = max(0, time)
        heapq.heappush(cost, (time+delta, delta+base, base))
    return ans

mountainHeight = 5
workerTimes = [1, 7]
process(mountainHeight, workerTimes)

### [统计重新排列后包含另一个字符串的子字符串数目 I](https://leetcode.cn/problems/count-substrings-that-can-be-rearranged-to-contain-a-string-i/)
### 题目
给你两个字符串word1和 word2。如果一个字符串x重新排列后，word2是重排字符串的前缀，那么我们称字符串x是合法的。请你返回word1中合法子字符串的数目。

### 分析
- 使用滑动窗口公式, 在遍历右端点时, 不断统计窗口中的字母出现次数, 当窗口满足word2的所有字母时, 可以开始缩小左窗口了. 在缩小时每缩小一次即可把答案+1，因此此时的[left, right]子串就是一个有效排列, 并且[left, right], [left, right+1], ..... [left, n-1]都是有效的排列, 因此每次缩小范围时需要对ans += n - right
- 使用collections.Counter可以快速对可迭代对象进行数量统计, 还可以进行大小统计.count1 >= count2 会以count1中每个键值作为基准去检查对应count2中的值是否满足>=关系, 如果count2中不存在键则值默认为0，遍历完count1中所有键值对后如果都满足>=关系则返回true。


In [ ]:
from collections import Counter

def process(word1, word2):
    n = len(word1)
    word1_dict = Counter()
    word2_dict = Counter(word2)
    left = 0   # 滑动窗口左端点
    ans = 0 
    # 滑动窗口右端点
    for right, char in enumerate(word1):
        word1_dict[char] += 1
        # 此时窗口中为[left, right]

        while word2_dict <= word1_dict:   # word2中所有出现字母数字都<=窗口中的
            ans += n-right   # [left, right], [left, right+1], ..... [left, n-1]
            word1_dict[word1[left]] -= 1
            left += 1
        # 此时[left, right] 已经不满足包含word2的结果了

    return ans


word1 = "abcabc"
word2 = "abc"
process(word1, word2)

### [统计重新排列后包含另一个字符串的子字符串数目 II](https://leetcode.cn/problems/count-substrings-that-can-be-rearranged-to-contain-a-string-ii/)
### 题目
给你两个字符串word1和 word2。如果一个字符串x重新排列后，word2是重排字符串的前缀，那么我们称字符串x是合法的。请你返回word1中合法子字符串的数目。

注意: 必须实现一个线性复杂度的解法。

### 分析
- 上一题的代码滑动窗口的时间复杂度为O(n), 而别忘了在比较word2_dict <= word1_dict时是根据哈希表挨个比较的, 因此这里的时间复杂度是26(最多26个字母), 因此上一题中时间复杂度为O(26n)
- 我们考虑while word2_dict <= word1_dict这一行代码, 这里我们窗口中每次增加一个字母后, 却需要去比较26个字母的大小关系, 实际上我们只用比较增加的这一个字母是否会影响到我们的窗口收缩。因此我们可以使用一个标识符, 记录word1窗口中是否全部大于word2_dict中的值

In [ ]:
from collections import Counter

def process(word1, word2):
    n = len(word1)
    word2_dict = Counter(word2)
    tag_count = len(word2_dict)
    left = 0   # 滑动窗口左端点
    ans = 0 
    # 滑动窗口右端点
    for right, char in enumerate(word1):
        word2_dict[char] -= 1
        # 此时窗口中为[left, right]
        if word2_dict[char] == 0:  # char减到0了, 说明窗口中的字母已经完全匹配了
            tag_count -= 1

        while tag_count == 0:   # 如果tag_count减少到0说明窗口中的所有字母已经能覆盖住word2中所有字母
            # 如果移除的字母是在word2中, 则意味着可能窗口中的字符会不满足条件
            if word1[left] in word2_dict:  
                word2_dict[word1[left]] += 1
                if word2_dict[word1[left]] > 0:  # 说明窗口中的字母已经不满足word2了
                    tag_count += 1
            left += 1
        ans += left   # [0, right], [1, right], ..... [left-1, right]都是符合条件的

        # 此时[left, right] 已经不满足包含word2的结果了

    return ans

## [419周赛](https://leetcode.cn/contest/weekly-contest-419/)

### [计算子数组的 x-sum II](https://leetcode.cn/problems/find-x-sum-of-all-k-long-subarrays-ii/description/)
### 题目
给你一个由n个整数组成的数组nums，以及两个整数k和x。数组的x-sum计算按照以下步骤进行:
- 统计数组中所有元素的出现次数。
- 仅保留出现次数最多的前x个元素的每次出现。如果两个元素的出现次数相同，则数值较大的元素被认为出现次数更多。

计算结果数组的和。注意: 如果数组中的不同元素少于x个，则其 x-sum 是数组的元素总和。

返回一个长度为n-k+1的整数数组answer，其中answer[i]是子数组nums[i: i + k] (长度为k的子数组) 的x-sum。


### 分析
- 暴力的方式: 遍历到i时, 将i-1元素从counter中去除, 将i+k-1元素加入counter中, 然后计算counter中出现次数前x的元素和, 即可完成answer[i]的计算。相当于是在使用窗口滑动, 每次滑动k个元素, 计算当前窗口内出现次数前x的元素和。但由于每次滑动后都需要对counter进行重新排序获取前x次出现次数元素, 因此时间复杂度太高
- 由于我们需要记录前x出现的元素, 我们考虑每当遍历到i时, 进出的元素实际上只会影响到两个元素的出现次数, 从而影响到counter的排序, 但是实际上counter的排序也只会影响到2个元素, 我们考虑有没有什么方式将这个排序的消耗变得最小: 使用两个顺序列表(自动维护列表元素的排列)
    - L: 记录出现频率前x的元素(如果频率相同, 则数值较大的元素更大)
    - R: 记录窗口中其他的元素
    - 由于需要进行多元素的排序, 因此L和R中的元素是(num出现次数, num)

- 我们要想实现上述行为需要抽象出四个方法:
    - add(num): 将num加入L或R中(放入窗口中), 如果L非空, 且num的出现次数比L中最小的要大, 则将num加入L, 否则加入R
    - remove(num): 将num从L或R中移除(移出窗口), 如果num在L中, 则将其移出L, 否则将其移出R
    - R2L: 将R中的最大元素移入L中, 保证L中元素是前x个出现次数最大的元素(平衡)
    - L2R: 将L中的最小元素移出L, 移入R中, 保证R中元素是窗口中其他元素(平衡)
    


In [ ]:
from sortedcontainers import SortedList
from collections import defaultdict

def process(nums, k: int, x: int):
    cnt = defaultdict(int)
    L = SortedList()   # 保存 tuple (出现次数，元素值)
    R = SortedList()
    n = len(nums)
    sum_l = 0          # L 的元素和
    answer = [0] * (n-k+1)

    def add(val: int) -> None:
        if cnt[val] == 0:  # 防止出现次数为0的元素加入L和R中
            return 
        p = (cnt[val], val)
        if L and p > L[0]:  # p 比 L 中最小的还大
            nonlocal sum_l
            sum_l += p[0] * p[1]
            L.add(p)
        else:
            R.add(p)

    def remove(val: int) -> None:
        if cnt[val] == 0:  # 如果cnt中val的出现次数已经是0了, 不能再减少了
            return 
        p = (cnt[val], val)
        if p in L:
            nonlocal sum_l
            sum_l -= p[0] * p[1]
            L.remove(p)
        else:
            R.remove(p)

    def L2R() -> None:
        nonlocal sum_l
        p = L[0]         # 找到L中最小的
        sum_l -= p[0] * p[1]
        L.remove(p)
        R.add(p)

    def R2L() -> None:
        nonlocal sum_l
        p = R[-1]        # 找到R中最大的
        sum_l += p[0] * p[1]
        R.remove(p)
        L.add(p)
    
    for r, in_ in enumerate(nums):
        # 移动元素前需要将in_旧状态从L和R中清除
        remove(in_)
        cnt[in_] += 1
        add(in_)
        
        l = r - k + 1   # 出去的num坐标
        if l < 0:       # 说明没有num需要出去
            continue
        
        # 此处的窗口中才是[0 ~ k-1]的元素
        while len(L) < x and R != []:  # 如果L中个数小于x，且R中还有元素，则将R中最大的元素加入L
            R2L()
        while len(L) > x and L != []:  # 如果L中个数大于x，且L中还有元素，则将L中最小的元素加入R
            L2R()

        # 记录下answer对应位置的答案
        answer[l] = sum_l

        # 移除 out
        out = nums[l]
        remove(out)   # 移动元素前需要将out旧状态从L和R中清除
        cnt[out] -= 1
        add(out)      # 移动元素后，将新状态加入L和R中

    return answer
            


### [ 第 K 大的完美二叉子树的大小](https://leetcode.cn/problems/k-th-largest-perfect-subtree-size-in-binary-tree/description/)
### 题目
给你一棵二叉树的根节点root和一个整数k。返回第k大的完美二叉子树的大小，如果不存在则返回-1。

完美二叉树: 又称为满二叉树, 是指所有叶子节点都在同一层级的树，且每个父节点恰有两个子节点。

### 分析:
- 对于满二叉树, 我们有两种判断方式: 1、根据左右子树的高度以及左右子树的节点个数来判断(2^hight - 1 = 树节点个数)。 2、根据左右子树的高度，以及左右子树是否同时为满二叉树来判断(只有当左右子树同时为满二叉树并且左右子树高度相同, 此时当前树才为满二叉树)。
- 由于本题中只用得到第k大的子树节点大小, 实际上我们只用在遍历树的过程中记录下为完美二叉树的树的高度即可(满二叉树的高度可以得到节点个数)。因此我们的递归函数只需要返回当前树的高度+是否为满二叉树即可。(实际是实现时, 当树不为满二叉树时高度返回-1即可)




In [ ]:
# 判断方式一: 
class Info():
    def __init__(self, hight, number):
        self.hight = hight
        self.count = number

# 递归函数 这个递归函数也可以用来计算一个未知二叉树的节点个数
def process(root,k):
    ans = []
    # 找到满二叉数
    def allTree(head):
        if head == None:
            return Info(0, 0)

        leftInfo = allTree(head.left)
        rightInfo = allTree(head.right)

        hight = max(leftInfo.hight, rightInfo.hight) + 1
        l = leftInfo.count + rightInfo.count + 1
        if (2**hight - 1) == l:   # 判断条件: 总节点数 等于 高度*2-1
            ans.append(l)
        return Info(hight, l)
    
    allTree(root)
    ans.sort(reverse=True)
    return ans[k-1] if len(ans) >= k else -1 


In [ ]:
# 判断方式二:
def process(root,k):
    ans = []

    def dfp(node):
        if node == None:
            return 0    # 说明高度为0
        left_hight = dfp(node.left)
        right_hight = dfp(node.right)

        if left_hight < 0 or right_hight < 0:  # 说明左子树/右子树有一个不为满二叉树
            return -1
        if left_hight != right_hight:   # 左右子树高度不相等, 也肯定不是满二叉树
            return -1
        # 来到这里说明当前树为满二叉树
        cur_hight = left_hight + 1
        ans.append(cur_hight)

        return cur_hight
        
    dfp(root)
    if k > len(ans):
        return -1
    ans.sort()
    return  (1<<ans[-k]) - 1   # 第k大高度, 得到第k大的节点个数


### [统计能获胜的出招序列数](https://leetcode.cn/problems/count-the-number-of-winning-sequences/description/)
### 题目
Alice和Bob正在玩一个幻想战斗游戏，游戏共有 n 回合，每回合双方各自都会召唤一个魔法生物：火龙（F）、水蛇（W）或地精（E）。每回合中，双方 同时 召唤魔法生物，并根据以下规则得分：
- W > F; F > E; E > W 获胜方得分+1
- 如果双方召唤相同的生物，那么两个玩家都不会获得分数。

给你一个字符串s，包含n个字符'F'、'W'和'E'，代表Alice每回合召唤的生物序列。Bob的出招序列未知，但保证Bob不会在连续两个回合中召唤相同的生物。如果在n轮后Bob获得的总分严格大于Alice的总分，则Bob战胜Alice。返回Bob可以用来战胜Alice的不同出招序列的数量。

### 分析:
- 很明显需要采用动态规划来做, 常规定义dp[i][diff]为截止到i位置Bob比Alice的积分差为diff(如果我们分别定义A的积分和B的积分那么转移方程就会很复杂, 由于有存在双方不得分的情况, 也不能简单的只定义B的积分, 因此需要定义为双方分数差)的情况数。但由于规则要求不能连续出招相同, 因此我们还需要一个维度记录下最截至i时最后一次出招。综上所述最终的dp定义为: 
    - dp[i][diff][0]: 为截止到i位置Bob比Alice的积分差为diff, 且最后一位出牌为F的序列数量
    - dp[i][diff][1]: 为截止到i位置Bob比Alice的积分差为diff, 且最后一位出牌为W的序列数量
    - dp[i][diff][2]: 为截止到i位置Bob比Alice的积分差为diff, 且最后一位出牌为E的序列数量
    - F,W,E 分别记作 0,1,2; 因此最后的比分规则为 1 > 0; 2 > 1; 0 > 2;
- 当Alice出0时(出其他时类似), 我们每次遍历F,W,E, 也就遍历Bob遍历第i次出招的情况:
    - 要得到diff的分差一共有3种情况: 这回和平局 + 前一回合得分为 diff-1 这回合获胜 + 前一回合得分为diff+1 这回合输掉。但是由于限定了本次出牌的顺序因此之会有其中一种可能出现
    - dp[i][diff][0]: (dp[i-1][diff][1]+dp[i-1][diff][2]) + 0(本回合出0是不可能获胜的) + 0(本回合出0是不可能输掉的)
    - dp[i][diff][1]: 0(本回合可能平局) + (dp[i-1][diff-1][0]+dp[i-1][diff-1][2]) + 0(本回合出1不可能输掉)
    - dp[i][diff][2]: 0 + 0 + (dp[i-1][diff][1] + dp[i-1][diff][0])
- 需要注意的是这里的diff可能为负数, 因此我们的第二维度长度应该为 2 * n + 1, 需要表示出-n和n的得分, 因此每次在计算diff维度是应该是使用n+diff作为下标。
- 当我们遍历到i时, 实际上j的可能取值只有【-i-1, i+1】因此遍历时的范围应该是range(-(i+1), i+2). 因为下标i时已经进行了i+1局
- 最终我们的答案应该是 dp[n][ n+diff > 0][0,1,2]的和


In [ ]:
def process(s):
    MOD = 1_000_000_007
    n = len(s)
    dp = [[ [0, 0, 0] for __ in range(2*n+2)] for _ in range(n)]  # 第二维度时2*n+2是因为要表示-n分以及+n分
    mp = {'F': 0, 'W': 1, 'E': 2}
    # 初始化
    if mp[s[0]] == 1:
        dp[0][n-1][0] = 1
        dp[0][n+0][1] = 1
        dp[0][n+1][2] = 1
    if mp[s[0]] == 0:
        dp[0][n+0][0] = 1
        dp[0][n+1][1] = 1
        dp[0][n-1][2] = 1
    if mp[s[0]] == 2:
        dp[0][n+1][0] = 1
        dp[0][n-1][1] = 1
        dp[0][n+0][2] = 1

    for i in range(1, n):
        a_car = mp[s[i]]
        for diff in range(-(i+1), i+2):
            if a_car == 0:
                dp[i][n+diff][0] = dp[i-1][n+diff][1] + dp[i-1][n+diff][2] 
                dp[i][n+diff][1] = dp[i-1][n+diff-1][0] + dp[i-1][n+diff-1][2]
                dp[i][n+diff][2] = dp[i-1][n+diff+1][0] + dp[i-1][n+diff+1][1]
            elif (a_car == 1):
                dp[i][n+diff][0] = dp[i-1][n+diff+1][1] + dp[i-1][n+diff+1][2] 
                dp[i][n+diff][1] = dp[i-1][n+diff][0] + dp[i-1][n+diff][2]
                dp[i][n+diff][2] = dp[i-1][n+diff-1][0] + dp[i-1][n+diff-1][1]
            elif (a_car == 2):
                dp[i][n+diff][0] = dp[i-1][n+diff-1][1] + dp[i-1][n+diff-1][2] 
                dp[i][n+diff][1] = dp[i-1][n+diff+1][0] + dp[i-1][n+diff+1][2]
                dp[i][n+diff][2] = dp[i-1][n+diff][0] + dp[i-1][n+diff][1]
    ans = 0
    for j in range(1, n+1):
        ans += dp[n-1][n+j][0] + dp[n-1][n+j][1] + dp[n-1][n+j][2]

    return ans % MOD

s = "FFF"
process(s)

## [421周赛](https://leetcode.cn/contest/weekly-contest-421/)

### [数组的最大因子得分](https://leetcode.cn/problems/find-the-maximum-factor-score-of-array/description/)
### 题目
给你一个整数数组 nums。因子得分定义为数组所有元素的最小公倍数（LCM）与最大公约数（GCD）的 乘积。在最多移除一个元素的情况下，返回nums的最大因子得分。注意：单个数字的 LCM 和 GCD 都是其本身，而 空数组 的因子得分为 0。

### 分析:
- 一组数据的最大公倍数/最小公因数是两两相求之后得到, 例如[a,b,c]需要先计算a,b的最大公倍数/最小公因数, 再用结果与c计算
- 遍历求解, 每次移除一个元素后求列表的最大公倍数 * 最小公因数 最后得到结果

In [ ]:
import math

def gcd_lcm_of_list(num_list):
    # 初始化最大公约数为列表中的第一个数
    gcd_val = num_list[0]
    # 初始化最小公倍数为列表中的第一个数
    lcm_val = num_list[0]

    for num in num_list[1:]:
        # 计算当前最大公约数和新数的最大公约数
        gcd_val = math.gcd(gcd_val, num)
        # 计算当前最小公倍数和新数的最小公倍数
        lcm_val = math.lcm(lcm_val, num)

    return gcd_val, lcm_val

def process(nums):
    n = len(nums)
    if n == 1:
        return nums[0] ** 2
    l, g = gcd_lcm_of_list(nums)
    res = l * g
    for i in range(n):
        l_tmp, g_tmp = gcd_lcm_of_list(nums[:i] + nums[i+1:])
        if l_tmp * g_tmp > res:
            res = l_tmp * g_tmp
        
    return res


### [字符串转换后的长度 I](https://leetcode.cn/problems/total-characters-in-string-after-transformations-i/description/)
### 题目
给你一个字符串s和一个整数t，表示要执行的转换次数。每次 转换 需要根据以下规则替换字符串 s 中的每个字符：
- 如果字符是 'z'，则将其替换为字符串 "ab"。
- 否则，将其替换为字母表中的下一个字符。例如，'a' 替换为 'b'，'b' 替换为 'c'，依此类推。

返回恰好执行t次转换后得到的字符串的长度。由于答案可能非常大，返回其对 109 + 7 取余的结果。

### 分析:
- 经发现每t = 26会增加相同长度的单词个数, 并且增加的单词是原来对应单词的+1。但注意原版对应单词是z, 那么该前25次转化后会得到yyb, 第26次转化会得到zzab, 相当于除了会贡献一个自身z以外还会贡献ab. 因此每轮转换时针对原版为z的单词需要特殊处理
- 由于在遍历字典时会更改相关字典的数值, 因此需要用copy复制字典, 否则出现相互影响
- 如果t不满26, 那么只有一部分的单词会会导致转化为ab(即导致长度+1), 我们只用找出相关的单词个数即可

In [ ]:
def process(s, t):
    MOD = 1_000_000_000 + 7
    n = len(s) 
    ans = n  # 字符串初始长度为s长度
    word_dict = {chr(i):0 for i in range(97, 123)}   # 导致长度+1还需要多少t: 单词个数
    for word in s:
        word_dict[word] += 1
    lun = t // 26
    yu = t % 26

    for _ in range(lun):   
        ans += ans  # 每一轮会增加原本字符串的长度
        word_dict_copy = word_dict.copy()
        for word, value in word_dict_copy.items():
            if value != 0:
                if word == "z":  # 如果为z
                    ans += value # 会额外增加value个长度
                    word_dict["a"] += value   # "a"的数量+1
                    word_dict["b"] += value   # "b"的数量+1
                else:
                    word_dict[chr(ord(word)+1)] += value
    # 有多少z
    for i in range(yu):
        ans += word_dict[chr(ord("z")-i)]  # 这些字母会被转化到"z"从而增加最终字符长度
    return ans%MOD


s = "jqktcurgdvlibczdsvnsg"   # 79033769
t = 7517
# s = "yz"
# t = 26
process(s, t)

### [字符串转换后的长度 II](https://leetcode.cn/problems/total-characters-in-string-after-transformations-ii/)
### 题目
给你一个由小写英文字母组成的字符串s，一个整数t表示要执行的转换次数，以及一个长度为26的数组nums。每次转换需要根据以下规则替换字符串s中的每个字符：
- 将 s[i] 替换为字母表中后续的 nums[s[i] - 'a'] 个连续字符。例如，如果 s[i] = 'a' 且 nums[0] = 3，则字符 'a' 转换为它后面的 3 个连续字符，结果为 "bcd"。
- 如果在步骤一中转换超过了 'z'，则回绕到字母表的开头。例如，如果s[i] = 'y'且nums[24] = 3，则字符 'y' 转换为它后面的3个连续字符，结果为 "zab"。

返回恰好执行t次转换后得到的字符串的长度。由于答案可能非常大，返回其对 109 + 7 取余的结果。

### 分析:
- 与上一题类似, 该题用nums[diff]规定了每个字母的替换规则, 如果nums[diff] = 1则代表只用自身+1转换, 如果nums[diff] = 2则还需要继续往后转换
- 我们需要探索一个字母经过26次变换后, 会产生多少个新字母, 以及新字母会产生多少个更新的字母(因为中途产生的新字母会继续产生新的字母)

In [ ]:
def process(s, t, nums):
    MOD = 1_000_000_000 + 7
    n = len(s) 
    lun_sum = sum(nums)
    ans = n  # 字符串初始长度为s长度
    word_dict = {chr(i):0 for i in range(97, 123)}   # 导致长度+1还需要多少t: 单词个数
    
    lun = t // 26
    yu = t % 26

    for _ in range(lun):   # 每一轮
        ans += lun_sum  # 会增加一轮长度
        word_dict_copy = word_dict.copy()
        for word, value in word_dict_copy.items():
            if value != 0:  # key = # 如果为z  (ord("z") - ord(word)+1)
                ans += nums[ord(word) - ord("a")]
                for i in range(1, nums[word - "a"]+1): # 增加的字母
                    next_word = chr(ord(word) + i)
                    if next_word > "z":
                        next_word = chr(ord(next_word) - ord("z") + ord("a") - 1)
                    word_dict[(ord("z") - ord(next_word) + 1)] += value
    
    for word in s:
        word_dict[(ord("z") - ord(word)+1)] += 1
    
    # 有多少z
    for i in range(1, yu+1):
        key = word_dict[i]
        word = ord("z") + 1 - key
        ans += nums[word - "a"]
    return ans%MOD


s = "abcyy"
t = 2
nums = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2]
# s = "yz"
# t = 26
process(s, t, nums)

In [ ]:
nums[ord("c") - ord("a")]

In [ ]:
word_dict = {chr(i):0 for i in range(97, 123)}   # 导致长度+1还需要多少t: 单词个数
word_dict

In [ ]:
next_word = chr(ord("z") + 1)
chr((ord(next_word) - ord("z") + ord("a") - 1))

## [xxx周赛]()

### [问题1]()
### 题目


### 分析:

### [问题2]()
### 题目


### 分析:

### [问题3]()
### 题目


### 分析:

## [xxx周赛]()

### [问题1]()
### 题目


### 分析:

### [问题2]()
### 题目


### 分析:

### [问题3]()
### 题目


### 分析:

## [xxx周赛]()

### [问题1]()
### 题目


### 分析:

### [问题2]()
### 题目


### 分析:

### [问题3]()
### 题目


### 分析:

## [xxx周赛]()

### [问题1]()
### 题目


### 分析:

### [问题2]()
### 题目


### 分析:

### [问题3]()
### 题目


### 分析: